## Imports

In [2]:
import pandas as pd
import requests
import json
import os

## Get financial statements

In [4]:
# читаем выгрузку из реестра
d1 = pd.read_excel('./данные/data1.xlsx', header=8)
d2 = pd.read_excel('./данные/data2.xlsx', header=8)

In [80]:
# обрабатываем случай 'один продукт - несколько компаний'
otm1 = d1[d1['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].str.replace(';', '').str.split('\n').sum()

otm2 = d2[d2['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].str.replace(';', '').str.split('\n').sum()

In [92]:
# обрабатываем случай 'один продукт - одна компания'
oto1 = d1[~d1['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].astype(str).values

oto2 = d2[~d2['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].astype(str).values

In [ ]:
# получаем список уникальных организаций
inns = list(set(otm1) | set(otm2) | set(oto1) | set(oto2))
api_key = ''

# получаем данные фин отчетности, локально сохраняем
for inn in inns:
    x = requests.get(f'https://api.checko.ru/v2/finances?key={api_key}&inn={inn}')
    with open(f'./reports/{inn}.txt', 'w') as f:
        json.dump(x.json(), f)

## Parse financial statements

In [174]:
# путь до фин отчетности
root = './reports/'
# годы, за которые смотрим фин результат
years = set(['2020', '2021', '2022', '2023'])

companies = pd.DataFrame()

for filename in os.listdir(root):
    with open(root + filename, 'r') as f:
        data = json.load(f)
        
    # для исследования нужна только выручка, ее код 2110
    revenue = {}

    for year in data['data']:
        if year in years and '2110' in data['data'][year]:
            revenue[year] = data['data'][year]['2110']
    
    if len(years - set(revenue.keys())) > 0:
        for year in years - set(revenue.keys()):
            revenue[year] = 0
    
    if 'company' in data:
        data['company'].update(revenue)
        if companies.empty:
            companies = pd.DataFrame(data['company'], index=[0])
        else:
            companies = pd.concat([companies, pd.DataFrame(data['company'], index=[0])])

In [176]:
companies.to_csv('./parsed_data.csv', index=False)

## Prepare registry

In [5]:
d1

,Регистрационный номер,Наименование ПО,Класс ПО,Дата внесения,Сайт с документацией по установке и эксплуатации ПО,Поддержка работы пользователей с ограничениями по зрению,Поддержка работы пользователей с ограничениями по слуху,Соответствие ПП №325 от 23.03.2017,Программное обеспечение относится к сфере искусственного интеллекта,Наименование (ФИО) правообладателя,Идентификационный номер (ИНН),Номер решения уполномоченного органа о включении сведений о ПО в реестр,Дата решения уполномоченного органа о включении сведений о ПО в реестр,Номер заявления о включении,Дата регистрации заявления о включении,Номер гос. регистрации,Дата гос. регистрации,Ссылка на стоимость
0,24144,ПО «Система мониторинга поисковых систем Crab»,Информационные системы для решения специфическ...,2024-09-23,https://dldev.ru/crab/documentation/,NaN,NaN,NaN,NaN,"ООО ""ПРОГРАММНЫЕ РЕШЕНИЯ - ПРО""",5401964160,б/н,2024-09-23,310423,2024-08-13,NaN,NaN,https://dldev.ru/crab/tarify/
1,24130,Аналитическая система «Делометрика»,Средства аналитической обработки в реальном вр...,2024-09-23,http://delometrica.ru/,NaN,NaN,NaN,NaN,"ООО ""УК Деломедика""",5018206146,б/н,2024-09-23,310485,2024-08-14,2024618604,15.04.2024,http://delometrica.ru/
2,24122,VirtualProxMox Special Edition (SE v.3.xx),Средства виртуализации;\nВстроенные системные ...,2024-09-23,https://digitalva.org/projects/proxmox,NaN,NaN,NaN,NaN,"ООО ""ЦВА""",7727487213,б/н,2024-09-23,310117,2024-08-09,2022662723,07.07.2022,https://digitalva.org/projects/proxmox
3,23992,Модуль обмена 1С:Бухгалтерия-DodoIS,Инструменты извлечения и трансформации данных ...,2024-09-06,https://denvic.ru/service/programmirovanie/nas...,NaN,NaN,NaN,NaN,"ООО ""Денвик-Систем""",1101144201,б/н,2024-09-06,309705,2024-08-02,2024619188,22.04.2024,https://denvic.ru/service/programmirovanie/nas...
4,23956,Система логической репликации данных из MySQL ...,Средства управления базами данных,2024-09-06,http://prof-it.com.ru/software.html,NaN,NaN,NaN,NaN,"ООО ""ПРОФ-ИТ""",9707026553,б/н,2024-09-06,309200,2024-07-25,NaN,NaN,http://prof-it.com.ru/software.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,104,СУБД Postgres Pro,Средства управления базами данных,2016-03-18,http://postgrespro.ru/doc,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПОСТ...",7729445882,112,2016-03-18,57182,2016-02-11,2019661679,05.09.2019,https://postgrespro.ru/products/postgrespro/en...
710,89,МойОфис Частное Облако,Серверное и связующее программное обеспечение;...,2016-03-18,https://myoffice.ru/products/cloud/,NaN,NaN,NaN,NaN,"ООО ""НОВЫЕ ОБЛАЧНЫЕ ТЕХНОЛОГИИ""",7703807270,112,2016-03-18,55729,2016-01-22,2018665258,03.12.2018,https://myoffice.ru/products/cloud/
711,88,МойОфис Профессиональный,Офисные пакеты;\nСредства хранения данных;\nСе...,2016-03-18,https://myoffice.ru/products/professional/,NaN,NaN,Да,NaN,"ООО ""НОВЫЕ ОБЛАЧНЫЕ ТЕХНОЛОГИИ""",7703807270,112,2016-03-18,55727,2016-01-22,2018665963,11.12.2018,https://myoffice.ru/products/professional/
712,37,RAIDIX,Средства хранения данных,2016-02-20,https://www.raidix.ru/products/raidix4/,NaN,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЭЙД...",7801546203,63,2016-02-20,58392,2016-02-16,NaN,NaN,https://www.raidix.ru/products/raidix4/


In [6]:
d2

,Регистрационный номер,Наименование ПО,Класс ПО,Дата внесения,Сайт с документацией по установке и эксплуатации ПО,Поддержка работы пользователей с ограничениями по зрению,Поддержка работы пользователей с ограничениями по слуху,Соответствие ПП №325 от 23.03.2017,Программное обеспечение относится к сфере искусственного интеллекта,Наименование (ФИО) правообладателя,Идентификационный номер (ИНН),Номер решения уполномоченного органа о включении сведений о ПО в реестр,Дата решения уполномоченного органа о включении сведений о ПО в реестр,Номер заявления о включении,Дата регистрации заявления о включении,Номер гос. регистрации,Дата гос. регистрации,Ссылка на стоимость
0,24144,ПО «Система мониторинга поисковых систем Crab»,Информационные системы для решения специфическ...,2024-09-23,https://dldev.ru/crab/documentation/,NaN,NaN,NaN,NaN,"ООО ""ПРОГРАММНЫЕ РЕШЕНИЯ - ПРО""",5401964160,б/н,2024-09-23,310423,2024-08-13,NaN,NaN,https://dldev.ru/crab/tarify/
1,24094,Система поддержки принятия врачебных решений п...,Программное обеспечение для решения отраслевых...,2024-09-23,https://bars-medical.ru/SPPVR,NaN,NaN,NaN,NaN,"ООО ""БАРС-МЕДИЦИНА""",1655336853,б/н,2024-09-23,309710,2024-08-02,NaN,NaN,https://bars-medical.ru/SPPVR
2,24055,НАЗЕМНЫЙ ПУНКТ УПРАВЛЕНИЯ ПОЛЕЗНЫМИ НАГРУЗКАМИ...,Средства мониторинга и управления;\nСредства п...,2024-09-23,https://silentwings.ru/,NaN,NaN,NaN,NaN,"ООО ""ГК Тихие Крылья""",7802703755,б/н,2024-09-23,309711,2024-08-02,2024613521,29.05.2024,https://silentwings.ru/
3,23990,Система поддержки принятия решений по здоровом...,Информационные системы для решения специфическ...,2024-09-06,https://healthheuristics.org/hh_diet_docs/,NaN,NaN,NaN,Да,"ООО ""Союз Спорт и здоровье""",9703005344,б/н,2024-09-06,309657,2024-08-01,2024663839,12.06.2024,https://docs.diet.hh-ai-serv.ru/oferta.pdf
4,23968,"""ЭнергоБИТ""",Средства поддержки принятия решений (DSS),2024-09-06,https://bit76.ru/solutions/energy,NaN,NaN,NaN,Да,"ООО ""БИТ""",7603044459,б/н,2024-09-06,309348,2024-07-29,2024617538,03.04.2024,https://bit76.ru/solutions/energy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,225,BI-Sphere,Средства поддержки принятия решений (DSS);\nСр...,2016-03-18,https://teleformis.ru/web/guest/bi-sphere,NaN,NaN,NaN,NaN,"Общество с ограниченной ответственностью ""Теле...",7705663948,112,2016-03-18,56267,2016-02-11,NaN,NaN,NaN
965,92,Логика СЭД. БП. СПО,Средства управления информационными ресурсами ...,2016-03-18,https://blogic.ru/products/docflow-solutions/b...,NaN,NaN,NaN,NaN,Общество с ограниченной ответственностью «Логи...,7735571356,112,2016-03-18,56879,2016-02-05,NaN,NaN,NaN
966,59,БАРС.Мониторинг - Транспорт,Программное обеспечение для оформления транспо...,2016-02-20,https://bars.group/upravlenie-regionom/monitor...,NaN,NaN,NaN,NaN,"АО ""БАРС ГРУП""",1655251590,63,2016-02-20,57820,2016-02-16,NaN,NaN,https://bars.group/upravlenie-regionom/monitor...
967,57,БАРС.Мониторинг - Эффективность Управления,Средства поддержки принятия решений (DSS),2016-02-20,https://bars.group/upravlenie-regionom/effekti...,NaN,NaN,NaN,NaN,"АО ""БАРС ГРУП""",1655251590,63,2016-02-20,57823,2016-02-16,NaN,NaN,https://bars.group/upravlenie-regionom/effekti...
